In [1]:
import os
import json

import numpy as np
import pandas


from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, InputLayer, LeakyReLU, Reshape, Flatten
from keras.layers.pooling import MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D

from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam

import librosa

Using TensorFlow backend.


In [2]:
def load_index(fold_id, key='train', n=None):
    
    with open(os.path.join(DESTDIR, 'experiment_{:02d}.json'.format(fold_id)), 'r') as fdesc:
        index = json.load(fdesc)[key]
    
    perm = np.random.permutation(len(index))
    if n is not None:
        perm = perm[:n]
        
    index = [index[_] for _ in perm]
    
    return index

In [3]:
def preprocess(X):
    return librosa.logamplitude(X**2, ref_power=np.max, top_db=80)


def load_data(index, n_harm=1):
    
    X, Y = [], []
    for item in index:
        with np.load(os.path.join(DESTDIR, 'features', '{}.npz'.format(item['filename']))) as data:
            if n_harm is None:
                n_harm = data['C'].shape[0]
            X.append(preprocess(data['C'][:n_harm]))
            Y.append(item['classID'])
    
    X = np.asarray(X).swapaxes(3, 1)
    Y = np.asarray(Y)
    
    return X, Y

In [4]:
DESTDIR = '/home/bmcfee/working/minst/'

In [5]:
FOLD_ID = 1

In [6]:
index = load_index(FOLD_ID)

In [7]:
test_index = load_index(FOLD_ID, key='test')

In [35]:
N_HARM = 4

In [36]:
X, Y = load_data(index, n_harm=N_HARM)

In [37]:
Xt, Yt = load_data(test_index, n_harm=N_HARM)

In [38]:
model = Sequential()

model.add(InputLayer(input_shape=X.shape[1:], name='input'))
model.add(BatchNormalization())

model.add(Conv2D(8, 5, (4 - X.shape[3] + 1) * 9, bias=False))

model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(16, 5, 5,  bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

#model.add(MaxPooling2D(pool_size=(5, 5)))

#model.add(Conv2D(64, 3, model.output_shape[2], bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))


#model.add(AveragePooling2D(pool_size=(model.output_shape[1], 1)))
model.add(MaxPooling2D(pool_size=(model.output_shape[1], 1)))
model.add(Flatten())

model.add(Dense(12, activation='softmax'))



In [39]:
model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [40]:
weight_fn = '/home/bmcfee/working/minst/models_{}.hdf5'.format(N_HARM)

In [41]:
history = model.fit(X, to_categorical(Y, nb_classes=12),
                    callbacks=[EarlyStopping('val_loss', patience=10),
                               ModelCheckpoint(weight_fn, save_best_only=True)],
                    validation_split=0.25, batch_size=32, shuffle=True,
                    nb_epoch=30)

Train on 26610 samples, validate on 8870 samples
Epoch 1/30
26610/26610 [==============================] - 28s - loss: 0.4452 - acc: 0.8549 - val_loss: 0.3164 - val_acc: 0.8843
Epoch 2/30
26610/26610 [==============================] - 27s - loss: 0.1268 - acc: 0.9611 - val_loss: 0.1351 - val_acc: 0.9554
Epoch 3/30
26610/26610 [==============================] - 27s - loss: 0.0778 - acc: 0.9752 - val_loss: 0.0745 - val_acc: 0.9744
Epoch 4/30
26610/26610 [==============================] - 27s - loss: 0.0596 - acc: 0.9813 - val_loss: 0.0683 - val_acc: 0.9772
Epoch 5/30
26610/26610 [==============================] - 27s - loss: 0.0448 - acc: 0.9855 - val_loss: 0.0914 - val_acc: 0.9703
Epoch 6/30
26610/26610 [==============================] - 27s - loss: 0.0381 - acc: 0.9881 - val_loss: 0.0675 - val_acc: 0.9770
Epoch 7/30
26610/26610 [==============================] - 27s - loss: 0.0349 - acc: 0.9882 - val_loss: 0.0917 - val_acc: 0.9688
Epoch 8/30
26610/26610 [==============================]

In [42]:
model.load_weights(weight_fn)

In [16]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

1
3417/3417 [==============================] - 0s     


[3.2804923124539425, 0.59555165323556636]

In [25]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

2
3417/3417 [==============================] - 1s     


[2.6730064784779266, 0.64062042701373434]

In [34]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

3
3417/3417 [==============================] - 1s     


[3.2844954945031475, 0.61018437223892164]

In [44]:
print(N_HARM)
model.evaluate(Xt, to_categorical(Yt))

4
3417/3417 [==============================] - 2s     


[2.6120286197176723, 0.60608721092530848]